In [1]:
import re
import graphlab as gl
from graphlab.toolkits.feature_engineering import TFIDF, QuadraticFeatures, FeatureHasher

# put in the path to the kaggle data
PATH_TO_JSON = "docs_prod_02/"
PATH_TO_TRAIN_LABELS = "input/train.csv"
PATH_TO_TEST_LABELS = "input/sampleSubmission.csv"

gl.canvas.set_target('ipynb')

[WARNING] This Python session does not appear to be running in an interactive IPython Notebook. Use of the 'ipynb' target may behave unexpectedly or result in errors.


In [234]:
files = gl.SFrame.read_csv('input/filenames.csv', header=False, column_type_hints=[str, int])
files.rename({'X1': 'id', 'X2': 'size'})
files['id'] = files['id'].apply(lambda x: str(x.split('_')[0] ))

PROGRESS: Finished parsing file /mnt/data/input/filenames.csv
PROGRESS: Parsing completed. Parsed 404076 lines in 0.173065 secs.


In [269]:
train_cv = gl.load_sframe('dataset_train_cv')
train_train = gl.load_sframe('dataset_train_train')

In [268]:
train_train = train_train.join(files, on='id', how='left')
train_cv = train_cv.join(files, on='id', how='left')

RuntimeError: Runtime Exception. Cancelled by user.

In [210]:
train_train['link_entropy'] = train_train['ahref'].apply(href_entropy)
train_cv['link_entropy'] = train_cv['ahref'].apply(href_entropy)

In [252]:
2 ** 14

16384

In [273]:
hash_encoder = gl.feature_engineering.create(train_train, FeatureHasher(features = ['tfidf'], num_bits=15, 
                                                                  output_column_name='tfidf_hashed'))
train_train = hash_encoder.transform(train_train)
train_cv = hash_encoder.transform(train_cv)

RuntimeError: Runtime Exception. Column name tfidf does not exist.

In [274]:
train_train.head()

sponsored,id,ahref,atext,img_avg,img_b_avg,img_b_cnt,img_cnt
0,2459274,"[#, #, #, #, #, #, #, /terms_of_use.php?ref=re ...","[register, sign in,continue with facebook, ...",0.0,0.0,0,14
0,2459283,"[http://web-marketing-consultants.com/, http ...","[, home, about us,services, video making, ...",0.0,0.0,0,33
0,2459284,"[http://www.flyfishdiary.com, #, http://www.fl ...","[, , home, equipment, flyfishing, fly tying, n ...",0.0,0.0,0,19
0,2459293,[/article/2947905/security/mozilla-blocks-all- ...,"[mozilla blocks all flashin firefox, 10 most ...",0.0,0.0,0,28
0,245932,[http://abcnews.go.com/International/iran-nucl ...,"[iran nuclear deal, plutonew horizons, el chapo ...",0.0,0.0,0,32
0,2459356,"[http://dailypicdump.com,http://dailypicdump.c ...","[, images, videos, gifs,to ferment and serve, ...",0.0,0.0,0,14
0,2459371,"[http://www.schleckysilberstein.com/, http://w ...","[, schlecky silbersteinune source d inspirat ...",0.0,0.0,0,8
0,2459374,"[http://www.sedentario.org/, https://dl.dropbo ...","[sedentrio & hiperativo,, duquian, internet, ...",0.0,0.0,0,58
0,245938,"[http://www.triveditestimonials.com, http://ww ...","[, home, about us, videotestimonial, fear from ...",0.0,0.0,0,11
0,2459396,"[//imgur.com, /hot, /new,/topic/Funny, ...","[, most viral, usersubmitted, funny, ...",0.0,0.0,0,1


In [6]:
train_cv[train_cv['script_avg'] > 1000000][0:3]

Columns:
	sponsored	int
	id	str
	ahref	list
	atext	list
	img_avg	float
	img_b_avg	float
	img_b_cnt	int
	img_cnt	int
	img_srcs	list
	misc_button	int
	misc_div	int
	misc_input	int
	misc_li	int
	misc_link	int
	misc_meta	int
	par	list
	script_avg	float
	script_b_avg	float
	script_b_cnt	int
	script_cnt	int
	script_srcs	list
	style_avg	float
	style_b_avg	float
	style_b_cnt	int
	style_cnt	int
	style_srcs	list
	title	str
	text_clean	str
	num_links	int
	num_links_non_short	int
	num_links_http	int
	num_clean_chars	int
	num_clean_chars_title	int
	quadratic_features	dict
	bow	dict
	tfidf	dict

Rows: 3

Data:
+-----------+---------+-------+-------+---------+-----------+-----------+---------+
| sponsored |    id   | ahref | atext | img_avg | img_b_avg | img_b_cnt | img_cnt |
+-----------+---------+-------+-------+---------+-----------+-----------+---------+
|     0     | 1028264 |   []  |   []  |   0.0   |    0.0    |     0     |    0    |
|     0     | 1918101 |   []  |   []  |   0.0   |    0.0    |     0     |    0    |
|     0     | 2445367 |   []  |   []  |   0.0   |    0.0    |     0     |    0    |
+-----------+---------+-------+-------+---------+-----------+-----------+---------+
+----------+-------------+----------+------------+---------+-----------+-----------+
| img_srcs | misc_button | misc_div | misc_input | misc_li | misc_link | misc_meta |
+----------+-------------+----------+------------+---------+-----------+-----------+
|    []    |      0      |    0     |     0      |    0    |     0     |     1     |
|    []    |      0      |    0     |     0      |    0    |     1     |     1     |
|    []    |      0      |    0     |     0      |    0    |     0     |     2     |
+----------+-------------+----------+------------+---------+-----------+-----------+
+-----+------------+--------------+--------------+------------+-----+
| par | script_avg | script_b_avg | script_b_cnt | script_cnt | ... |
+-----+------------+--------------+--------------+------------+-----+
|  [] | 1994182.0  |  1994182.0   |      1       |     1      | ... |
|  [] | 1007724.0  |  1007724.0   |      1       |     1      | ... |
|  [] | 1081838.0  |  1081838.0   |      1       |     1      | ... |
+-----+------------+--------------+--------------+------------+-----+
[3 rows x 36 columns]

In [69]:
from urlparse import urlparse
import re
import collections

In [121]:
social_domain_names = [
    'twitter.com',
    'facebook.com',
    'google.com',
    'pinterest.com',
    'linkedin.com',
    'reddit.com',
    'olark.com',
    'disqus.com',
    'apple.com',
    'yelp.com',
    'imgur.com',
    'instagram.com',
    'youtube.com',
    'tumblr.com',
    'feedburner.com',
    ]

foreign_domain_pattern = re.compile(r'^[^\.]{2,3}\.[^\.]{2}$')

In [209]:
def not_social_domain(domain):
    return domain not in social_domain_names
    
def href2domain(href):
    try:
        host = urlparse(href).netloc
        num_pieces = host.count('.') + 1
        if num_pieces <= 2:
            return host
        shortened_host = host[(host.find('.')+1):]
        if num_pieces == 3: 
            if foreign_domain_pattern.match(shortened_host):
                return host
            else:
                return shortened_host
        more_shortened_host = shortened_host[(shortened_host.find('.')+1):]
        if foreign_domain_pattern.match(more_shortened_host):
            return shortened_host
        else:
            return more_shortened_host        
    except Exception as inst:
        print inst
        print href
        return 'bad_bad_domain'
        
def hrefs2domains(hrefs):
    return [href2domain(href) for href in hrefs]

def proper_domains(hrefs):
    domains = hrefs2domains(hrefs)
    domains = filter(not_social_domain, domains)
    #domains = filter(lambda d: d is not '', domains)
    return domains

def entropy(probs):
    return -1.0 * sum([prob*log(prob, 2) for prob in probs])

def href_entropy(hrefs):
    domains = proper_domains(hrefs)
    total = len(domains)
    if total == 0:
        return 4.0
    else:
        counts = collections.Counter(domains).values()
        if len(counts) == 1:
            return 0.0
        else:
            return entropy([1.0*count/total for count in counts])    
        
def href_entropy_alternative2(hrefs):
    domains = proper_domains(filter(lambda d: d is not '', hrefs))
    total = len(domains)
    if total == 0:
        return 1.0
    else:
        #counts = collections.Counter(domains).values()
        counter = collections.Counter(domains)
        default_host_count = counter.get('', 0)
        del counter['']
        if len(counter) <= 1:
            return 0.0
        else:
            items = counter.items()
            items.sort(key=lambda (x,y): -y)
            new_c1 = default_host_count + items[0][1]
            counts = [new_c1] + [item[1] for item in items[1:]]
            return entropy([1.0*count/total for count in counts])    

def href_entropy_alternative(hrefs):
    domains = proper_domains(hrefs)
    total = len(domains)
    if total == 0:
        return 1.0
    else:
        #counts = collections.Counter(domains).values()
        items = collections.Counter(domains).items()
        items.sort(key=lambda (x,y): -y)
        if len(items) == 1:
            return 0.0
        else:
            c1 = items[0][1]
            c2 = items[1][1]
            new_c1 = int((c1 - c2)*0.3 + c2)
            counts = [new_c1] + [item[1] for item in items[1:]]
            items = (items)
            return entropy([1.0*count/total for count in counts])    

In [171]:
entropy([1.0])

-0.0

In [ ]:
train_train['ahref'].apply(lambda ahrefs: 
                              filter(None, 
                                     list(set([href2domain(ahref) for ahref in ahrefs]))
                                    )
                          ).show()

In [104]:
hrefs_0 = train_train[train_train['sponsored'] == 0]['ahref']

In [98]:
hrefs = train_train[train_train['sponsored'] == 1]['ahref']

In [167]:
type(hrefs)

graphlab.data_structures.sarray.SArray

In [169]:
gl.SArray([href_entropy(x) for x in hrefs_0[0:10000]]).show()

Invalid IPv6 URL
http://store.heartmath.org']


In [168]:
gl.SArray(([href_entropy(x) for x in hrefs[0:10000]])).show()

In [212]:
gl.SArray([href_entropy(x) for x in hrefs_0[0:10000]]).show()

Invalid IPv6 URL
http://store.heartmath.org']


In [213]:
gl.SArray([href_entropy(x) for x in hrefs[0:10000]]).show()

In [204]:
this_hrefs = hrefs_0[110]
href_entropy(this_hrefs), proper_domains(this_hrefs), this_hrefs

(0.20405611985266403,
 ['',
  '',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  '',
  'enikos.gr',
  'enikos.gr',
  '',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  '',
  'enikos.gr',
  '',
  '',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  '',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  'enikos.gr',
  '',
  'spiritup.gr',
  'enik

In [84]:
train_train[train_train['ahref'].apply(lambda ahrefs: any(['send' in urlparse(ahref).netloc for ahref in ahrefs]))]['ahref'].show()

RuntimeError: Runtime Exception. Traceback (most recent call last):
  File "<ipython-input-84-2cb0b7b23f38>", line 1, in <lambda>
  File "/usr/lib64/python2.7/urlparse.py", line 143, in urlparse
    tuple = urlsplit(url, scheme, allow_fragments)
  File "/usr/lib64/python2.7/urlparse.py", line 191, in urlsplit
    raise ValueError("Invalid IPv6 URL")
ValueError: Invalid IPv6 URL


In [41]:
urlparse('http://www.google.com/hello/there.php;paramshere?id=12&m=6#inde')

ParseResult(scheme='http', netloc='www.google.com', path='/hello/there.php', params='paramshere', query='id=12&m=6', fragment='inde')

In [5]:
train_cv[train_cv['style_avg'] > 900000][0:3]

Columns:
	sponsored	int
	id	str
	ahref	list
	atext	list
	img_avg	float
	img_b_avg	float
	img_b_cnt	int
	img_cnt	int
	img_srcs	list
	misc_button	int
	misc_div	int
	misc_input	int
	misc_li	int
	misc_link	int
	misc_meta	int
	par	list
	script_avg	float
	script_b_avg	float
	script_b_cnt	int
	script_cnt	int
	script_srcs	list
	style_avg	float
	style_b_avg	float
	style_b_cnt	int
	style_cnt	int
	style_srcs	list
	title	str
	text_clean	str
	num_links	int
	num_links_non_short	int
	num_links_http	int
	num_clean_chars	int
	num_clean_chars_title	int
	quadratic_features	dict
	bow	dict
	tfidf	dict

Rows: 2

Data:
+-----------+---------+-------+-------+---------+-----------+-----------+---------+
| sponsored |    id   | ahref | atext | img_avg | img_b_avg | img_b_cnt | img_cnt |
+-----------+---------+-------+-------+---------+-----------+-----------+---------+
|     0     | 1085576 |   []  |   []  |   0.0   |    0.0    |     0     |    0    |
|     0     | 3625546 |   []  |   []  |   0.0   |    0.0    |     0     |    0    |
+-----------+---------+-------+-------+---------+-----------+-----------+---------+
+----------+-------------+----------+------------+---------+-----------+-----------+
| img_srcs | misc_button | misc_div | misc_input | misc_li | misc_link | misc_meta |
+----------+-------------+----------+------------+---------+-----------+-----------+
|    []    |      0      |    0     |     0      |    0    |     0     |     1     |
|    []    |      0      |    0     |     0      |    0    |     0     |     1     |
+----------+-------------+----------+------------+---------+-----------+-----------+
+-----+------------+--------------+--------------+------------+-----+
| par | script_avg | script_b_avg | script_b_cnt | script_cnt | ... |
+-----+------------+--------------+--------------+------------+-----+
|  [] |    0.0     |     0.0      |      0       |     0      | ... |
|  [] |    0.0     |     0.0      |      0       |     0      | ... |
+-----+------------+--------------+--------------+------------+-----+
[2 rows x 36 columns]

In [8]:
train_cv[train_cv['num_links'] > 8000][0:3]

Columns:
	sponsored	int
	id	str
	ahref	list
	atext	list
	img_avg	float
	img_b_avg	float
	img_b_cnt	int
	img_cnt	int
	img_srcs	list
	misc_button	int
	misc_div	int
	misc_input	int
	misc_li	int
	misc_link	int
	misc_meta	int
	par	list
	script_avg	float
	script_b_avg	float
	script_b_cnt	int
	script_cnt	int
	script_srcs	list
	style_avg	float
	style_b_avg	float
	style_b_cnt	int
	style_cnt	int
	style_srcs	list
	title	str
	text_clean	str
	num_links	int
	num_links_non_short	int
	num_links_http	int
	num_clean_chars	int
	num_clean_chars_title	int
	quadratic_features	dict
	bow	dict
	tfidf	dict

Rows: 3

Data:
+-----------+---------+-------------------------------+
| sponsored |    id   |             ahref             |
+-----------+---------+-------------------------------+
|     0     |  561078 | [#mw-head, #p-search, /wik... |
|     0     | 2217476 | [/accounts, #content, http... |
|     0     | 1143746 | [#mw-head, #p-search, /wik... |
+-----------+---------+-------------------------------+
+-------------------------------+---------+-----------+-----------+---------+
|             atext             | img_avg | img_b_avg | img_b_cnt | img_cnt |
+-------------------------------+---------+-----------+-----------+---------+
| [navigation, search, list ... |   0.0   |    0.0    |     0     |    5    |
| [lost your password or use... |   0.0   |    0.0    |     0     |    8    |
| [navigation, search, a ser... |   0.0   |    0.0    |     0     |   129   |
+-------------------------------+---------+-----------+-----------+---------+
+-------------------------------+-------------+----------+------------+---------+
|            img_srcs           | misc_button | misc_div | misc_input | misc_li |
+-------------------------------+-------------+----------+------------+---------+
| [//upload.wikimedia.org/wi... |      0      |    47    |     4      |   1949  |
| [/i/header/masthead/flagma... |      2      |   2036   |     53     |   581   |
| [//upload.wikimedia.org/wi... |      0      |   545    |     4      |   1841  |
+-------------------------------+-------------+----------+------------+---------+
+-----------+-----------+-------------------------------+------------+--------------+
| misc_link | misc_meta |              par              | script_avg | script_b_avg |
+-----------+-----------+-------------------------------+------------+--------------+
|     15    |     3     | [a train song is a song re... |   400.4    |    444.8     |
|     5     |     17    | [skip to main content, sun... |   315.7    |    558.8     |
|     15    |     3     | [the netherlands despite i... |   498.9    |    554.2     |
+-----------+-----------+-------------------------------+------------+--------------+
+--------------+------------+-----+
| script_b_cnt | script_cnt | ... |
+--------------+------------+-----+
|      9       |     10     | ... |
|      22      |     39     | ... |
|      9       |     10     | ... |
+--------------+------------+-----+
[3 rows x 36 columns]

In [28]:
train_cv[train_cv['misc_meta'] > 1000][0:3]

Columns:
	sponsored	int
	id	str
	ahref	list
	atext	list
	img_avg	float
	img_b_avg	float
	img_b_cnt	int
	img_cnt	int
	img_srcs	list
	misc_button	int
	misc_div	int
	misc_input	int
	misc_li	int
	misc_link	int
	misc_meta	int
	par	list
	script_avg	float
	script_b_avg	float
	script_b_cnt	int
	script_cnt	int
	script_srcs	list
	style_avg	float
	style_b_avg	float
	style_b_cnt	int
	style_cnt	int
	style_srcs	list
	title	str
	text_clean	str
	num_links	int
	num_links_non_short	int
	num_links_http	int
	num_clean_chars	int
	num_clean_chars_title	int
	quadratic_features	dict
	bow	dict
	tfidf	dict

Rows: 2

Data:
+-----------+---------+-------------------------------+
| sponsored |    id   |             ahref             |
+-----------+---------+-------------------------------+
|     0     | 1050915 | [http://ad4.netshelter.net... |
|     0     | 2311640 | [/Lightbox, /Member/SignUp... |
+-----------+---------+-------------------------------+
+-------------------------------+---------+-----------+-----------+---------+
|             atext             | img_avg | img_b_avg | img_b_cnt | img_cnt |
+-------------------------------+---------+-----------+-----------+---------+
| [, home, android news, new... |   0.0   |    0.0    |     0     |   102   |
| [lightbox 0 , register, lo... |   0.0   |    0.0    |     0     |   104   |
+-------------------------------+---------+-----------+-----------+---------+
+-------------------------------+-------------+----------+------------+---------+
|            img_srcs           | misc_button | misc_div | misc_input | misc_li |
+-------------------------------+-------------+----------+------------+---------+
| [http://ad4.netshelter.net... |      0      |   143    |     4      |   129   |
| [https://s.ftcdn.net/r/v20... |      5      |   944    |     29     |   102   |
+-------------------------------+-------------+----------+------------+---------+
+-----------+-----------+-------------------------------+------------+--------------+
| misc_link | misc_meta |              par              | script_avg | script_b_avg |
+-----------+-----------+-------------------------------+------------+--------------+
|     16    |    1409   | [xiaomi has released a lot... |   370.7    |    595.7     |
|     15    |    1107   | [need help? +1 888 674 229... |   423.5    |    564.3     |
+-----------+-----------+-------------------------------+------------+--------------+
+--------------+------------+-----+
| script_b_cnt | script_cnt | ... |
+--------------+------------+-----+
|      23      |     37     | ... |
|      9       |     12     | ... |
+--------------+------------+-----+
[2 rows x 36 columns]

In [30]:
train_cv[train_cv['misc_li'] > 5000][0:3]

Columns:
	sponsored	int
	id	str
	ahref	list
	atext	list
	img_avg	float
	img_b_avg	float
	img_b_cnt	int
	img_cnt	int
	img_srcs	list
	misc_button	int
	misc_div	int
	misc_input	int
	misc_li	int
	misc_link	int
	misc_meta	int
	par	list
	script_avg	float
	script_b_avg	float
	script_b_cnt	int
	script_cnt	int
	script_srcs	list
	style_avg	float
	style_b_avg	float
	style_b_cnt	int
	style_cnt	int
	style_srcs	list
	title	str
	text_clean	str
	num_links	int
	num_links_non_short	int
	num_links_http	int
	num_clean_chars	int
	num_clean_chars_title	int
	quadratic_features	dict
	bow	dict
	tfidf	dict

Rows: 3

Data:
+-----------+---------+-------------------------------+
| sponsored |    id   |             ahref             |
+-----------+---------+-------------------------------+
|     0     | 1053066 | [//www.blogger.com/rearran... |
|     0     |  435609 | [http://www.jobsforuscitiz... |
|     0     |  903843 | [http://www.medyouth.com/,... |
+-----------+---------+-------------------------------+
+-------------------------------+---------+-----------+-----------+---------+
|             atext             | img_avg | img_b_avg | img_b_cnt | img_cnt |
+-------------------------------+---------+-----------+-----------+---------+
| [, artodyssey, artodyssey1... |   0.0   |    0.0    |     0     |    36   |
| [blog, contact, terms and ... |   0.0   |    0.0    |     0     |    1    |
| [home, , blogger, , , 10 d... |   0.0   |    0.0    |     0     |    56   |
+-------------------------------+---------+-----------+-----------+---------+
+-------------------------------+-------------+----------+------------+---------+
|            img_srcs           | misc_button | misc_div | misc_input | misc_li |
+-------------------------------+-------------+----------+------------+---------+
| [//img1.blogblog.com/img/i... |      0      |   101    |     0      |   5328  |
| [http://www.jobsforuscitiz... |      1      |    39    |     15     |   6655  |
| [//img1.blogblog.com/img/i... |      1      |   383    |     13     |   5899  |
+-------------------------------+-------------+----------+------------+---------+
+-----------+-----------+-------------------------------+------------+--------------+
| misc_link | misc_meta |              par              | script_avg | script_b_avg |
+-----------+-----------+-------------------------------+------------+--------------+
|     9     |     2     | [painting is silent poetry... |   682.9    |    1116.9    |
|     10    |     11    | [msysinc posted by msysinc... |   235.1    |    605.7     |
|     15    |     6     | [, posted by admin posted ... |   427.1    |    614.6     |
+-----------+-----------+-------------------------------+------------+--------------+
+--------------+------------+-----+
| script_b_cnt | script_cnt | ... |
+--------------+------------+-----+
|      11      |     18     | ... |
|      12      |     31     | ... |
|      50      |     72     | ... |
+--------------+------------+-----+
[3 rows x 36 columns]

In [275]:
features = [
            #'img_cnt',
            #'misc_button', 'misc_div', 'misc_input', 'misc_li', 'misc_meta',
            #'script_avg', 'script_cnt', 'script_b_avg', 'script_b_cnt',
            #'style_avg', 'style_cnt', 'style_b_avg', 'style_b_cnt',
            #'num_links', 'num_links_non_short', 'num_links_http', 'num_clean_chars', 'num_clean_chars_title',
            'link_entropy', 'size', 
            'quadratic_features',
            'tfidf_hashed'
            #'tfidf'
           ]
#features = ['tfidf']
features = ['tfidf_hashed']

In [265]:
clean_train_train = train_train[train_train['size'] < 1000000]
clean_train_cv = train_cv[train_cv['size'] < 1000000]

In [ ]:
model = gl.logistic_classifier.create(train_train, target='sponsored', 
                                      features=features,
                                      validation_set=train_cv,
                                      class_weights='auto',
                                      l2_penalty=0.00, l1_penalty=0.0)

In [241]:
clean_train_train.shape, train_train.shape

((283753, 38), (285698, 38))

In [242]:
283753.0/285698

0.9931921119503812

In [226]:
train_train[train_train['num_clean_chars'] < 30000].shape

(269652, 37)

In [233]:
train_train[train_train['num_clean_chars'] >= 550000]['sponsored'].show()